In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('train.csv')
df = df.set_index('id')
df.head()

,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,...,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
id,,,,,,,,,,,,,,,,,,,,,
22267382,Modern and Cozy Large Studio in Brooklyn,Modern large studio with new amenities and app...,Our place is a little quiet sanctuary in the h...,Modern large studio with new amenities and app...,none,"BAM, Barclays, Brooklyn City Point, Fort Green...",NaN,"Subway: 2,3,4,5,A,C,B,Q,G",Washer/Dryer Dishwasher Internet Gym Roof Top ...,"Depending on the time of your visit, I'll be h...",...,10.0,10.0,10.0,t,f,flexible,f,f,1,0.59
2473861,Royal Harlem TRIPLEX Home 5 Beds,Harlem is back and so gorgeous! Visit and expl...,Harlem is back and so gorgeous! Visit and expl...,Harlem is back and so gorgeous! Visit and expl...,none,HARLEM is a piece of real NY history overflowi...,HARLEM RESTAURANTS Red Rooster Harlem -- excel...,PUBLIC TRANSPORTATION: Conveniently near all p...,The WHOLE ENTIRE HOUSE,NaN,...,9.0,9.0,9.0,t,f,moderate,f,f,3,2.47
25079703,Sunny East Village Studio,"Clean, hip and well designed sun drenched East...",This is a rare East Village studio with it's h...,"Clean, hip and well designed sun drenched East...",none,East Village is one of the last remaining neig...,NaN,NaN,You'll have access to the entire space - it's ...,"Very responsive via phone call, text or email.",...,10.0,10.0,10.0,f,f,moderate,f,f,1,0.89
9342478,"Beautiful, airy, light-filled room","Private, spacious, comfortable room in 2-bed f...","Big closet, two big windows, tall ceiling and ...","Private, spacious, comfortable room in 2-bed f...",none,One block from Morgan L stop. Super cool area....,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,f,f,flexible,f,f,1,NaN
4866426,Private Room in Prime Brooklyn Spot,"Comfy, quiet and big private room in a three b...",This big old apartment that we love and take c...,"Comfy, quiet and big private room in a three b...",none,I absolutely love this neighborhood - right at...,Just a note about the space: The window in you...,Super convenient to almost all subway lines. A...,Your room has a very comfortable queen sized b...,"We are my husband Joaquin and I, our sweet new...",...,10.0,10.0,10.0,f,f,flexible,f,f,1,3.14


In [4]:
test = pd.read_csv('test.csv')
test = test.set_index('id')
test.head()

,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,...,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
id,,,,,,,,,,,,,,,,,,,,,
19307997,Super Lux 2BR in Downtown Manhattan,Prepare to be WOWED! This spectacularly bright...,"Top of the line Wolf and Sub-Zero appliances, ...",Prepare to be WOWED! This spectacularly bright...,none,NaN,NaN,NaN,NaN,electronic lock ensures 24 hour check-in,...,NaN,NaN,NaN,f,f,flexible,f,f,1,NaN
20176193,Vintage Eclectic Brownstone Pad in Brooklyn,"Ideal for romantic, creative types, this is an...","Not your typical New York abode, my apartment ...","Ideal for romantic, creative types, this is an...",none,Bed Stuy is a diverse historic neighborhood wi...,This is an actual unique living experience whe...,Close to buses and subways there is also free ...,"Entrance hallway, living room, bedroom, kitche...",I'm very social but understand the need for my...,...,10.0,10.0,10.0,f,f,flexible,f,f,1,1.48
19485371,Spacious Harlem Hideaway,"Postive Vibes . This is our Harlem tree house,...",The private room is very spacious and cozy. Th...,"Postive Vibes . This is our Harlem tree house,...",none,You are in a Cultural Haven full of restaurant...,We also keep cucumber water in the fridge feel...,"Train, uber or a taxi. (Extremely taxi accessi...","Private Room, Kitchen And Bathroom",Very open to communication. If you have any qu...,...,10.0,10.0,10.0,t,f,flexible,f,f,1,0.37
13079990,Spacius private room in Brooklyn,"Newly renovated apartment, its a 3 bedroom apa...","3 bedroom apartment, 1 full bathroom, living r...","Newly renovated apartment, its a 3 bedroom apa...",none,NaN,NaN,"There is the Mta 3 train Sutter stop, also the...",NaN,NaN,...,9.0,8.0,9.0,f,f,flexible,f,f,1,0.23
22339757,*Dg) Delightful Private Room 20 min to Manhattan,Hi my home is only 2 blocks from the subway st...,NaN,Hi my home is only 2 blocks from the subway st...,none,NaN,NaN,NaN,NaN,NaN,...,8.0,8.0,8.0,t,f,strict_14_with_grace_period,f,f,9,1.53


In [5]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# First try: cleaned features (not including bert analysis for language)

In [5]:
def clean1(df):
    cleaned = df.copy()
    cleaned['transit'] = 1-df['transit'].isna().astype(int) #there is transportation listed
    cleaned['host_is_superhost'] = (df['host_is_superhost'] == 't').astype(int)
    cleaned['host_has_profile_pic'] = (df['host_has_profile_pic'] == 't').astype(int)
    cleaned['host_identity_verified'] = (df['host_identity_verified'] == 't').astype(int)
    cleaned['instant_bookable'] = (df['instant_bookable'] == 't').astype(int)
    cleaned['is_business_travel_ready'] = (df['is_business_travel_ready'] == 't').astype(int)
    cleaned['require_guest_profile_picture'] = (df['require_guest_profile_picture'] == 't').astype(int)
    cleaned['require_guest_phone_verification'] = (df['require_guest_phone_verification'] == 't').astype(int)
    
    dict_cancellation_policy = {'flexible': 1, 'moderate': 2, 'super_strict_60': 7,
       'strict_14_with_grace_period': 5, 'super_strict_30': 6, 'long_term': 4,
       'strict': 3} #reference: https://www.airbnb.com/help/article/475
    cleaned['cancellation_policy'] = df['cancellation_policy'].apply(lambda x: dict_cancellation_policy[x])
    X1 = cleaned[['transit', 'host_is_superhost', \
                    'host_listings_count', 'accommodates', 'bathrooms', 'bedrooms', \
                    'beds', 'guests_included', 'minimum_nights', 'maximum_nights', 'number_of_reviews',\
                   'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness',
                    'review_scores_checkin', 'review_scores_communication', 'review_scores_location', \
                    'review_scores_value','instant_bookable','is_business_travel_ready', 'require_guest_profile_picture',\
                    'require_guest_phone_verification', 'calculated_host_listings_count', 'reviews_per_month']]
    
    return X1

In [6]:
cleaned = df.copy()

In [7]:
cleaned['transit'] = 1-df['transit'].isna().astype(int) #there is transportation provided
cleaned['host_is_superhost'] = (df['host_is_superhost'] == 't').astype(int)
cleaned['host_has_profile_pic'] = (df['host_has_profile_pic'] == 't').astype(int)
cleaned['host_identity_verified'] = (df['host_identity_verified'] == 't').astype(int)
cleaned['instant_bookable'] = (df['instant_bookable'] == 't').astype(int)
cleaned['is_business_travel_ready'] = (df['is_business_travel_ready'] == 't').astype(int)
cleaned['require_guest_profile_picture'] = (df['require_guest_profile_picture'] == 't').astype(int)
cleaned['require_guest_phone_verification'] = (df['require_guest_phone_verification'] == 't').astype(int)

In [8]:
dict_cancellation_policy = {'flexible': 1, 'moderate': 2, 'super_strict_60': 7,
       'strict_14_with_grace_period': 5, 'super_strict_30': 6, 'long_term': 4,
       'strict': 3} #reference: https://www.airbnb.com/help/article/475

In [9]:
cleaned['cancellation_policy'] = df['cancellation_policy'].apply(lambda x: dict_cancellation_policy[x])

In [10]:
X1 = cleaned[['transit', 'host_is_superhost', \
                    'host_listings_count', 'accommodates', 'bathrooms', 'bedrooms', \
                    'beds', 'guests_included', 'minimum_nights', 'maximum_nights', 'number_of_reviews',\
                   'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness',
                    'review_scores_checkin', 'review_scores_communication', 'review_scores_location', \
                    'review_scores_value','instant_bookable','is_business_travel_ready', 'require_guest_profile_picture',\
                    'require_guest_phone_verification', 'calculated_host_listings_count', 'reviews_per_month']]
y1 = cleaned['price']

In [11]:
X1_train, X1_val, y1_train, y1_val = train_test_split(X1, y1, test_size=0.25, shuffle=True)

In [12]:
from xgboost import XGBRegressor

In [13]:
xgb = XGBRegressor(base_score= 145.177291430616, booster='gbtree', tree_method = 'approx', grow_policy='lossguide') #max_depth=1000, n_estimators= 1000, grow_policy='lossguide'
xgb

XGBRegressor(base_score=145.177291430616, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy='lossguide',
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, predictor=None, random_state=None, ...)

In [14]:
xgb.fit(X1_train, y1_train)

XGBRegressor(base_score=145.177291430616, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy='lossguide',
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, predictor=None, random_state=None, ...)

In [15]:
# train accuracy - RMSE
sum((xgb.predict(X1_train) - y1_train)**2) / len(y1_train)

5687.524234312602

In [16]:
# val accuracy
sum((xgb.predict(X1_val) - y1_val)**2) / len(y1_val)

9839.626542027881

In [17]:
xgb1 = XGBRegressor(gamma=0.15) #max_depth=1000, n_estimators= 1000, grow_policy='lossguide'
xgb1.fit(X1, y1)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.15, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [18]:
test1 = clean1(test)
preds1 = xgb.predict(test1)

In [19]:
pd.DataFrame(index=test.index, data=preds1, columns=['Predicted']).to_csv('preds1.csv')

Later

$$# # drop quasi
# from sklearn.feature_selection import VarianceThreshold
# sel = VarianceThreshold(threshold=0.05)
# sel.fit(cleaned)$$



In [15]:
import lightgbm as lgb

OSError: dlopen(/Users/js/opt/anaconda3/lib/python3.9/site-packages/lightgbm/lib_lightgbm.so, 0x0006): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: <D21A7969-4567-3BC7-94ED-6A9E83AE9D78> /Users/js/opt/anaconda3/lib/python3.9/site-packages/lightgbm/lib_lightgbm.so
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file, not in dyld cache)

# ver2: use grid search to tune hyperparameters
- was not helpful at all

In [200]:
X2_train, X2_val, y2_train, y2_val = train_test_split(X1, y1, test_size=0.30, shuffle=True)

In [201]:
xgb = XGBRegressor(grow_policy='lossguide') 
xgb

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy='lossguide',
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, predictor=None, random_state=None, ...)

In [211]:
parameters = {
    'max_depth': range (2, 20, 1),
    'n_estimators': range(60, 220, 40),
    'tree_method': ['approx', 'auto', 'exact', 'hist'],
    'gamma': range(0, 1, 0.1)
    
}

In [237]:
grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=parameters,
    scoring = 'roc_auc',
    n_jobs = 10,
    cv = 5,
    verbose=True
)

In [240]:
grid_search.fit(X2_train, y2_train)

Fitting 5 folds for each of 360 candidates, totalling 1800 fits


/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 349, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. D

/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 349, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. D

/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 349, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. D

/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 349, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. D

...

/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 349, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. D

/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 349, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. D

/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 349, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. D

/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 349, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/Users/js/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. D

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=145.177291430616,
                                    booster='gbtree', callbacks=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy='lossguide',
                                    importance_type=None,
                                    interaction_constraints=No...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    

In [245]:
grid_search.best_estimator_.max_depth
grid_search.best_estimator_.n_estimators
grid_search.best_estimator_.tree_method




'approx'

In [243]:
# max depth = 2
# n_estimators = 60
# tree = approx